https://riptutorial.com/julia-lang/example/26313/guide

In [2]:
u = 42
f = x -> x^2
typeof(f)

var"#11#12"

宏会把接收的参数 **接收为表达式**, 如果参数为字面量，接收为字面量<br>
如下面的 `show(x)` 结果为 `:u`<br><br>

宏返回的表达式时，会执行

In [1]:
# Let's make our own @show macro:
macro log(x)
    show(x)
    println("\n")
    
    :(
      println( "Expression: ", x, " has value: ", $x )
    )
end

# @log(u)  # error 找不到Main.x
@macroexpand @log(u) 

:u



:(Main.println("Expression: ", Main.x, " has value: ", Main.u))

In [7]:
@macroexpand @log(f(42))

:(f(42))



:(Main.println("Expression: ", Main.x, " has value: ", Main.f(42)))

In [ ]:
macro log(x)
    show(x)
    println("\n")
    
    :(
      println( "Expression: ", $x, " has value: ", $x )
    )
end

@log(u)
@macroexpand @log(u) 

:u

Expression: 42 has value: 42
:u



:(Main.println("Expression: ", Main.u, " has value: ", Main.u))

In [ ]:
macro log(x)
    show(x)
    println("\n")
    
    :(
      println( "Expression: ", string($x), " has value: ", $x )
    )
end

@log(u)
@macroexpand @log(u) 

:u

Expression: 42 has value: 42
:u



:(Main.println("Expression: ", Main.string(Main.u), " has value: ", Main.u))

In [ ]:
macro log(x)
  show(x)
  println("\n")
  
  :(
    println( "Expression: ", string(x), " has value: ", $x )
  )
end

# @log(u)  # error 找不到Main.x
@macroexpand @log(u) 

:u



:(Main.println("Expression: ", Main.string(Main.x), " has value: ", Main.u))

In [8]:
macro log(x)
    show(x)
    println("\n")
    
    :(
      println( "Expression: ", $string(x), " has value: ", $x )
    )
  end
  
  # @log(u)  # error 找不到Main.x
  @macroexpand @log(u) 

:u



:(Main.println("Expression: ", (string)(Main.x), " has value: ", Main.u))

In [9]:
macro log(x)
    show(x)
    println("\n")
    
    :(
      println( "Expression: ", $(string(x)), " has value: ", $x )
    )
  end
  
  @log(u)
  @macroexpand @log(u) 

:u

Expression: u has value: 42
:u



:(Main.println("Expression: ", "u", " has value: ", Main.u))

In [10]:
@log(42)

println("\n-- --")
@log(:u)

println("\n-- --")
@log(f(42))

42

Expression: 42 has value: 42

-- --
:(:u)

Expression: :u has value: u

-- --
:(f(42))

Expression: f(42) has value: 1764


https://docs.juliacn.com/latest/manual/metaprogramming/#构建高级的宏

这是 Julia 的 @assert 宏的简化定义. 将其编写为函数是不可能的，因为函数能获取的只有条件的值而无法在错误信息中**显示计算出它的表达式**。宏是操作语句表达式的，所以可以。

In [1]:
macro assert(ex)
    show(ex)
    println("\n")
    show(string(ex))
    return :( $ex ? nothing : throw(AssertionError($(string(ex)))) ) 
    # string(ex)为什么要有 $ ? 原因在前述已经讲过
    # return :( $ex ? nothing : throw(AssertionError(string(ex))) ) # Error Main中没有ex
    # return :( $ex ? nothing : throw(AssertionError(string($ex))) ) # AssertionError: false
end

@assert (macro with 1 method)

https://riptutorial.com/julia-lang/example/26313/guide<br>

Q: Why the last `$`? A: It interpolates, i.e. forces Julia to eval that `string(ex)` as execution passes through the invocation of this macro. i.e. If you just run that code it won't force any evaluation. But the moment you do `assert(foo)` Julia will **invoke** this macro replacing its 'AST token/Expr' with whatever it returns, and the `$` will kick into action.<br><br>
问：为什么最后一个 `$` ？ 答：它插入，即强制Julia  `eval`   `string(ex)` 执行通过这个宏的调用。 也就是说，如果你只是运行代码，它不会强制执行任何求值。但是当您执行 `assert(foo)` 时，Julia将调用该宏，用它返回的任何内容替换其‘AST令牌/Expr’，并且 `$` 将开始行动。

In [2]:
@assert 1 == 1.0

:(1 == 1.0)

"1 == 1.0"

In [3]:
@assert 1 == 0

:(1 == 0)

"1 == 0"

AssertionError: AssertionError: 1 == 0

In [16]:
@macroexpand @assert 1 == 0

:(if 1 == 0
      nothing
  else
      Base.throw(Base.AssertionError("1 == 0"))
  end)

In [21]:
a=2; b=3
@macroexpand @assert a==b "a ($a) should equal b ($b)!"  
# @assert接收1个表达式，传入两个表达式 不对, @macroexpand展开的结果看不懂

:(if a == b
      nothing
  else
      Base.throw(Base.AssertionError(((Base.Main).Base.inferencebarrier((Base.Main).Base.string))("a ($(a)) should equal b ($(b))!")))
  end)

In [44]:
macro assert2(ex, msgs...)
    msg_body = isempty(msgs) ? ex : msgs[1]
    show(msg_body)
    println("\n")
    
    msg = string(msg_body)    
    show(msg)
    
    return :($ex ? nothing : throw(AssertionError($msg)))
end

@macroexpand @assert2 a == b

:(a == b)

"a == b"

:(if Main.a == Main.b
      Main.nothing
  else
      Main.throw(Main.AssertionError("a == b"))
  end)

In [45]:
@macroexpand @assert2 a==b "a should equal b!"

"a should equal b!"

"a should equal b!"

:(if Main.a == Main.b
      Main.nothing
  else
      Main.throw(Main.AssertionError("a should equal b!"))
  end)

In [24]:
typeof(:("a should equal b"))

String

In [25]:
dump(:("a ($a) should equal b ($b)!"))

Expr
  head: Symbol string
  args: Array{Any}((5,))
    1: String "a ("
    2: Symbol a
    3: String ") should equal b ("
    4: Symbol b
    5: String ")!"


In [47]:
a=2; b=3
@assert2 a==b "a ($a) should equal b ($b)!"

:("a ($(a)) should equal b ($(b))!")

"\"a (\$(a)) should equal b (\$(b))!\""

AssertionError: AssertionError: "a ($(a)) should equal b ($(b))!"

In [48]:
macro assert3(ex, msgs...)
    msg_body = isempty(msgs) ? ex : msgs[1]
    show(msg_body)
    println("\n")
    
    msg = msg_body    
    show(msg)
    
    
    return :($ex ? nothing : throw(AssertionError($msg)))
end

@macroexpand @assert3 a == b "a ($a) should equal b ($b)!"

:("a ($(a)) should equal b ($(b))!")

:("a ($(a)) should equal b ($(b))!")

:(if Main.a == Main.b
      Main.nothing
  else
      Main.throw(Main.AssertionError("a ($(Main.a)) should equal b ($(Main.b))!"))
  end)

In [49]:
a=2; b=3
@assert3 a==b "a ($a) should equal b ($b)!"

:("a ($(a)) should equal b ($(b))!")

:("a ($(a)) should equal b ($(b))!")

AssertionError: AssertionError: a (2) should equal b (3)!

https://stackoverflow.com/a/41103043

In [10]:
macro quoted(expression)

    show(expression)
    println("\n")

    show(QuoteNode(expression))
    println("\n")
    show(Expr(:quote, expression))
    println("\n")
    
    quote
        println("received expression: :(", QuoteNode(expression), ")")
        Expr(:quote, expression)
    end
end

x = 1

1

In [11]:
@macroexpand @quoted $x + 1

:($(Expr(:$, :x)) + 1)

:($(QuoteNode(:($(Expr(:$, :x)) + 1))))

:($(Expr(:quote, :($(Expr(:$, :x)) + 1))))



quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:12 =#
    Main.println("received expression: :(", Main.QuoteNode(Main.expression), ")")
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:13 =#
    Main.Expr(:quote, Main.expression)
end

In [18]:
macro quoted2(expression)

    show(expression)
    println("\n")

    show(QuoteNode(expression))
    println("\n")
    show(Expr(:quote, expression))
    println("\n")
    
    quote
        println("received expression: :(", $QuoteNode(expression), ")")
        $Expr(:quote, expression)
    end
end

x = 1

1

In [19]:
@macroexpand @quoted2 $x + 1

:($(Expr(:$, :x)) + 1)

:($(QuoteNode(:($(Expr(:$, :x)) + 1))))

:($(Expr(:quote, :($(Expr(:$, :x)) + 1))))



quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sZmlsZQ==.jl:12 =#
    Main.println("received expression: :(", (QuoteNode)(Main.expression), ")")
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sZmlsZQ==.jl:13 =#
    (Expr)(:quote, Main.expression)
end

In [39]:
macro quoted3(expression)

    show(expression)
    println("\n")

    show(QuoteNode(expression))
    println("\n")
    show(Expr(:quote, expression))
    println("\n")    
    
    quote
        println("received expression: :(", $(QuoteNode(expression)), ")")
        $(Expr(:quote, expression))
    end
end

x = 1

1

In [27]:
@macroexpand @quoted3 $x + 1

:($(Expr(:$, :x)) + 1)

:($(QuoteNode(:($(Expr(:$, :x)) + 1))))

:($(Expr(:quote, :($(Expr(:$, :x)) + 1))))



quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X44sZmlsZQ==.jl:12 =#
    Main.println("received expression: :(", $(QuoteNode(:($(Expr(:$, :x)) + 1))), ")")
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X44sZmlsZQ==.jl:13 =#
    Core._expr(:call, :+, Main.x, 1)
end

In [25]:
@quoted3 $x + 1

:($(Expr(:$, :x)) + 1)

:($(QuoteNode(:($(Expr(:$, :x)) + 1))))

:($(Expr(:quote, :($(Expr(:$, :x)) + 1))))

received expression: :($x + 1)


:(1 + 1)

In [33]:

quoted3_return_ex =
quote
    :(:(1 + 1))
end
eval(eval(quoted3_return_ex))

:(1 + 1)

In [34]:
macro eval_quote()
quote
    :(:(1 + 1))
end
end
@macroexpand @eval_quote

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X51sZmlsZQ==.jl:3 =#
    $(Expr(:copyast, :($(QuoteNode(:($(Expr(:quote, :(1 + 1)))))))))
end

In [35]:
@eval_quote

:($(Expr(:quote, :(1 + 1))))

In [36]:
macro eval_quote2()
    quote
        :(1 + 1)
    end
end
@macroexpand @eval_quote2

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X53sZmlsZQ==.jl:3 =#
    $(Expr(:copyast, :($(QuoteNode(:(1 + 1))))))
end

In [37]:
@eval_quote2

:(1 + 1)

In [41]:
macro quoted4(expression)

    show(expression)
    println("\n")

    show(QuoteNode(expression))
    println("\n")
    show(Expr(:quote, expression))
    println("\n")    
    
    quote
        println("received expression: :(", $(QuoteNode(expression)), ")")
        Expr(:quote, expression)
    end
end

x = 1

1

In [42]:
@macroexpand @quoted4 $x + 1

:($(Expr(:$, :x)) + 1)

:($(QuoteNode(:($(Expr(:$, :x)) + 1))))

:($(Expr(:quote, :($(Expr(:$, :x)) + 1))))



quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:12 =#
    Main.println("received expression: :(", $(QuoteNode(:($(Expr(:$, :x)) + 1))), ")")
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:13 =#
    Main.Expr(:quote, Main.expression)
end

In [43]:
@quoted4 $x + 1

:($(Expr(:$, :x)) + 1)

:($(QuoteNode(:($(Expr(:$, :x)) + 1))))

:($(Expr(:quote, :($(Expr(:$, :x)) + 1))))

received expression: :($x + 1)


UndefVarError: UndefVarError: `expression` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [44]:
@quoted3 :(x + 1)

:($(Expr(:quote, :(x + 1))))

:($(QuoteNode(:($(Expr(:quote, :(x + 1)))))))

:($(Expr(:quote, :($(Expr(:quote, :(x + 1)))))))

received expression: :(:(x + 1))


:($(Expr(:quote, :(x + 1))))

In [45]:
:($(Expr(:quote, :(x + 1)))) == :(:(x + 1))

true